<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/wikiWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##学習済みDoc2Vecモデル
https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/

ライブラリ
Gensim: 3.6.0

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from gensim.models import KeyedVectors
from pprint import pprint

#Gドライブ保存後の確認
model_dir = './drive/My Drive/NLP/wiki_tohoku.model'
model = KeyedVectors.load(model_dir)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 3.4MB/s 


In [126]:
import MeCab

def search_similar_texts(p_words):
  #パラメータから括弧が消えている？
  p_words = "[" + p_words + "]"
  try:
    print(model.similarity('[ポケモン]',p_words))
  except KeyError as error:
    pass

words = 'マツコ・デラックス'
inherent_words = '[' + words + ']'
  
#**************************************************************************
#そのまま対義化
#**************************************************************************
asitis_cng_words = ''
try:
  for i in range(5):
    rvs_wd = model.most_similar(positive=[inherent_words,u"逆"])[i][0]
    if inherent_words.replace('[',"").replace(']',"") != rvs_wd.replace('[',"").replace(']',""):
      asitis_cng_words = model.most_similar(positive=[inherent_words,u"逆"])[i][0]
      break
    else:
      asitis_cng_words = words + "(変換なし)"
except KeyError as error:
  #辞書に登録の無い単語の場合
  search_similar_texts(inherent_words)

#**************************************************************************
#形態素分析後に対義化
#**************************************************************************
word_cng_list = []
tokenizer = MeCab.Tagger("-Ochasen")
node = tokenizer.parseToNode(words)
  
while node:
  cut_wd = node.surface

  if node.feature.split(",")[0] == u"名詞":
    try:
      for i in range(5):
        rvs_wd = model.most_similar(positive=[cut_wd,u"逆"])[i][0]
        if cut_wd != rvs_wd.replace('[',"").replace(']',""):
          #◆結合
          word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
          break
    except KeyError as error:
      print("名詞キーエラー")
      #辞書に登録の無い単語の場合
      search_similar_texts(cut_wd)

  elif node.feature.split(",")[0] == u"形容詞":
    cut_wd = node.feature.split(",")[6]

    try:
      for i in range(5):
        rvs_wd = model.most_similar(positive=[cut_wd,u"逆"])[i][0]
        if cut_wd != rvs_wd.replace('[',"").replace(']',""):
          #◆結合
          word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
          break
    except KeyError as error:
      print("形容詞キーエラー")
      #辞書に登録の無い単語の場合
      search_similar_texts(cut_wd)

  elif node.feature.split(",")[0] == u"動詞":
    cu_wd = node.feature.split(",")[6]

    try:
      for i in range(5):
        rvs_wd = model.most_similar(positive=[cut_wd,u"逆"])[i][0]
        if cut_wd != rvs_wd.replace('[',"").replace(']',""):
          #◆結合
          word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
          break
    except KeyError as error:
      print("動詞キーエラー")
      #辞書に登録の無い単語の場合
      search_similar_texts(cut_wd)

  else:
    #◆結合
    word_cng_list.append(cut_wd.replace('[',"").replace(']',""))

  node = node.next

print(words)
print(asitis_cng_words)
print(word_cng_list)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


マツコ・デラックス
ボケ
['', 'ボケ', '・', 'ノーマル', '']


In [96]:
print(model.similarity('凶悪', '[サイドン]'))


0.036208138


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [92]:
#r'文字列'　→　正規表現。
#u'文字列'　→　日本語。Ver.2.7のとき必要。Ver.3では付けても付けなくてもよい。
try:
  #pprint(model.wv.most_similar(positive=[u'世界'],negative=[u"紫金山"]))
  pprint(model.most_similar(positive=[u'[サイドン]',u'逆']))
  #pprint(model.wv.most_similar(negative=[u'世界'], topn=10))
except KeyError as error:
  print(error)

[('[ベロリンガ]', 0.6900442242622375),
 ('modify', 0.6766635179519653),
 ('つるぎのまい', 0.676105260848999),
 ('ョウ', 0.673792839050293),
 ('[板チタン石]', 0.6733494997024536),
 ('spinalis', 0.6712069511413574),
 ('銀漢', 0.6707651615142822),
 ('[太極拳釋名]', 0.670268177986145),
 ('火遠理命', 0.6698230504989624),
 ('[ルクソン]', 0.6689685583114624)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
